In [ ]:
from bs4 import BeautifulSoup
import requests
import urllib
from urllib.parse import urlparse
import time
import re
import pandas as pd
import numpy as np
from datetime import datetime #추가

In [ ]:
# 검색어 결과 창에서 뉴스 링크 가져오기 (언론사 링크 + 네이버 링크)
# 네이버 뉴스는 최대 400페이지까지만 제공
def get_news_url(query, start_date, end_date) :
    page = 1
    cnt = 1
    url_list = []
    fail_pg_list = []

    while True :
        print(f'\r{cnt}페이지 크롤링 중입니다.', end='')
        url = f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={query}&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&cluster_rank=17&mynews=0&office_type=0&office_section_code=0&news_office_checked=&office_category=0&service_area=0&nso=so:r,p:from{start_date}to{end_date},a:all&start={page}"
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/100.0.48496.75"}

        # HTML 파싱
        response = requests.get(url,headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 링크 가져오기
        news_group = soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
        href_list = [a.attrs['href'] for a in news_group if 'href' in a.attrs]
        url_list.extend(href_list)
        #print(href_list)

        # 다음 페이지 버튼 활성화 여부
        btn_next = soup.select_one("div.sc_page > a.btn_next").attrs['aria-disabled']

        if btn_next is None :
            # None : 크롤링 접속 차단 -> 따로 하기
            fail_pg_list.append(page)
        elif btn_next == 'false' :
            pass
        elif btn_next == 'true' :
            print("="*25,f'{cnt}번 째가 마지막 페이지로 크롤링을 종료합니다',"="*25)
            break

        # 일시정지 - 크롤링 접속 차단 문제 해결
        if cnt % 9 == 0 :
            time.sleep(30)

        '''
        if cnt % 50 == 0:
            break
            print('page:',page)
        '''

        page += 10
        cnt += 1

    print("크롤링 실패한 페이지 리스트: ",fail_pg_list)

    return url_list

In [ ]:
# 네이버뉴스 링크만 추출하기
# ⭐ sid=106 은 연예 -> 씨젠, 에코프로와는 관련 X
# ⭐ sid=106 링크 크롤링 실패해서 링크 직접 검색해보니...링크가 달라져요....
# ⭐ 엔터주 검색 시, sid=106 링크 앞부분 수정 필요
def naver_news_url(url_list) :
    naver_urls = []
    naver_urls = [url for url in url_list if 'https://n.news.naver.com/' in url and 'sid=106' not in url]
    return naver_urls

In [ ]:
# 각 뉴스에서 제목, 일자, 본문 가져오기
def get_news_contents(naver_urls) :
    cnt = 1
    company = []
    titles = []
    dates = []
    contents = []
    links = []
    fail_urls = []

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/100.0.48496.75"}

    for url in naver_urls :
        response = requests.get(url,headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        if soup.select_one("div.media_end_head_title > h2.media_end_head_headline") == None :
            fail_urls.append(url)

        else :
            media = soup.select_one("div.media_end_head.go_trans > div.media_end_head_top > a > img.media_end_head_top_logo_img.light_type").get('title')
            title = soup.select_one("div.media_end_head_title > h2.media_end_head_headline").get_text()

            date = soup.select_one("div.media_end_head_info_datestamp_bunch > span").get_text()
            date = date.split()[0].rstrip('.')   #추가
            date = datetime.strptime(date, "%Y.%m.%d")   #추가
            cleaned_date = date.strftime("%Y-%m-%d")  #추가


            content = soup.select_one("#dic_area").get_text().strip("\n")
            cleaned_text = re.sub(r'[\n\t]', '', content)

            company.append(media)
            titles.append(title)
            dates.append(cleaned_date)  #추가
            contents.append(content)
            links.append(url)

        if cnt % 9 == 0 :
            time.sleep(40)
        #if cnt % 100 == 0 :
            #time.sleep(60)
        cnt += 1

    print("크롤링 실패한 링크 리스트: ",fail_urls)

    return company, titles, dates, contents, links

In [ ]:
import pandas as pd

def main(query, start_date, end_date) :
    url_list = get_news_url(query, start_date, end_date)
    naver_urls = naver_news_url(url_list)
    company, titles, dates, contents, links  = get_news_contents(naver_urls)
    data = {'언론사': company,'제목': titles, '작성일시': dates, '본문': contents, '링크' : links}
    df = pd.DataFrame(data)

    return df

In [ ]:
# 기본 변수 설정
data_final = pd.DataFrame()

while True :
    query = input("검색어를 입력해주세요 : ")

    if query == 'end' :
        break

    start_date = input("시작 일자를 입력해주세요(예 20230501) : ")
    end_date = input("종료 일자를 입력해주세요(예 20230501) : ")

    print("="*20,f"검색어 {query}에 대해 {start_date}에서 {end_date}까지의 기사 크롤링을 시작합니다.","="*20)

    df = main(query, start_date, end_date)

    data_final = pd.concat([data_final,df],axist=0)

In [ ]:
data_final.to_csv("./data/이차전지_230501_230731.csv",index = None)

In [ ]:
'''
# 제목, 일자, 본문 추출 테스트
url = "https://n.news.naver.com/mnews/article/665/0000001502?sid=101"
response = requests.get(url,headers=header)
soup = BeautifulSoup(response.text, 'html.parser')

title = soup.select_one("div.media_end_head_title > h2.media_end_head_headline").get_text()
date = soup.select_one("div.media_end_head_info_datestamp_bunch > span").get_text()
content = soup.select_one("#dic_area").get_text().strip("\n")
'''